In [20]:
import os, sys
import time
import datetime
import pandas as pd
import numpy as np
import math
from math import radians, cos, sin, asin, sqrt 
import random
import copy

class Dynamic_programming(object):
    
    def __init__(self,Regions,State,Action,End_step,Matching_Prob,Pickup_Prob,Dest_Prob,Travel_time):
        
        '''Param'''

        self.Regions=Regions

        self.State=State

        self.Action=Action

        self.End_step=End_step

        self.gamma=0.8

        '''Reward'''

        self.V_table={state:0.0 for state in self.State}

        self.Q_table={}

        for state in self.State:

            self.Q_table[state]={}

            for action in self.Action[state]:

                self.Q_table[state][action]=0.0

        '''Prob'''

        self.Matching_Prob=Matching_Prob
        
        self.Pickup_Prob=Pickup_Prob

        self.Dest_Prob=Dest_Prob
        
        '''Travel Time'''
        
        self.Travel_time=Travel_time

        '''Iteration'''

        self.Policy={state:state.split('_')[0] for state in self.State}

        self.Backwards()
        
    def Backwards(self):
        
        for t in range(self.End_step-2,-1,-1):
            
            print('*'*50)
            
            print('Current step: ',t)
            
            for reg in self.Regions:
                
                state=reg+'_'+str(int(t))
                
                for action in self.Action[state]:
                    
                     self.Q_table[state][action]= self.Compute_Q(state,action)
                        
                self.V_table[state]=max(self.Q_table[state].values())
                
                self.Policy[state]=max(self.Q_table[state], key=self.Q_table[state].get)
                    
    def Compute_Q(self,state,action):
        
        reg=state.split('_')[0]+'_'+state.split('_')[1]
        
        step=int(state.split('_')[-1])
        
        action_tep=step+int(self.Travel_time[reg][action])
        
        if action_tep<self.End_step:
        
            action_state=action+'_'+str(action_tep)

            p_m=self.Matching_Prob[action_state]

            '''Matching'''

            r_1=1

            if len(self.Pickup_Prob[action])!=0:

                for pickup_reg in self.Pickup_Prob[action]:

                    p_pickup=self.Pickup_Prob[action][pickup_reg]

                    if len(self.Dest_Prob[pickup_reg])!=0:

                        for dest_reg in self.Dest_Prob[pickup_reg]:

                            p_dest=self.Dest_Prob[pickup_reg][dest_reg]

                            dest_step=action_tep+int(self.Travel_time[action][pickup_reg])+int(self.Travel_time[pickup_reg][dest_reg])

                            dest_state=dest_reg+'_'+str(dest_step)

                            if dest_step<self.End_step:

                                r_1+=self.gamma**(action_tep-step)*(p_pickup*p_dest*self.V_table[dest_state])




            '''Not matching'''

            r_2=0

            r_2+=self.gamma*self.V_table[action_state]

            '''Expected reward'''

            r=r_1*p_m+(1-p_m)*r_2
            
        else:
            
            r=0.0
        
        return r

In [21]:
'''Load data'''

End_step=180


'''Zone-related data'''

Zone_list=np.load('./Data/NYC_Zones/Zone_list.npy',allow_pickle=True)

'''State Action'''

State=np.load('./Data/MDP/State.npy',allow_pickle=True)

Action=np.load('./Data/MDP/Action.npy',allow_pickle=True).item()

'''Probability'''

Matching_Prob=np.load('./Data/MDP/Matching_Prob.npy',allow_pickle=True).item()

Pickup_Prob=np.load('./Data/MDP/Pickup_Prob.npy',allow_pickle=True).item()

Dest_Prob=np.load('./Data/MDP/Dest_Prob.npy',allow_pickle=True).item()

'''Travel_time'''

Travel_time=np.load('./Data/MDP/Travel_time.npy',allow_pickle=True).item()


In [22]:
DP=Dynamic_programming(Zone_list,State,Action,End_step,Matching_Prob,Pickup_Prob,Dest_Prob,Travel_time)


**************************************************
Current step:  178
**************************************************
Current step:  177
**************************************************
Current step:  176
**************************************************
Current step:  175
**************************************************
Current step:  174
**************************************************
Current step:  173
**************************************************
Current step:  172
**************************************************
Current step:  171
**************************************************
Current step:  170
**************************************************
Current step:  169
**************************************************
Current step:  168
**************************************************
Current step:  167
**************************************************
Current step:  166
**************************************************
Current step:  165
********************

**************************************************
Current step:  60
**************************************************
Current step:  59
**************************************************
Current step:  58
**************************************************
Current step:  57
**************************************************
Current step:  56
**************************************************
Current step:  55
**************************************************
Current step:  54
**************************************************
Current step:  53
**************************************************
Current step:  52
**************************************************
Current step:  51
**************************************************
Current step:  50
**************************************************
Current step:  49
**************************************************
Current step:  48
**************************************************
Current step:  47
**********************************

In [25]:
np.save('./Data/MDP/Policy',DP.Policy)

np.save('./Data/MDP/Q_table',DP.Q_table)

np.save('./Data/MDP/V_table',DP.V_table)

